In [1]:
import numpy as np
import sys
import pandas as pd
import os
pi=np.pi
from numba import jit
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.layouts import column
output_notebook()
from bokeh.palettes import plasma
import pickle as pk

Loading BokehJS ...

In [2]:
#from calc_mie import mie_calc  
import PyMieScatt as ps

In [3]:
thetd=0
n_thetd=1
## testing purpose parameters
nwave=196#184
ngas=1
nrad= 40
igas=0
gas_name=["Mg2SiO4"]#"Al2O3","Cr", "Fe","H2O","MgSiO3","KCl","MnS","Na2S","NH3"]
wave=np.zeros(shape=(nwave))
qext=np.zeros(shape=(nwave,nrad,ngas))
qscat = np.zeros(shape=(nwave,nrad,ngas))
cos_qscat=np.zeros(shape=(nwave,nrad,ngas))
radius=np.zeros(shape=(nrad,ngas))
rup=np.zeros(shape=(nrad,ngas))
dr=np.zeros(shape=(nrad,ngas))
vrat = 2.2
rmin = 1e-5
pw = 1. / 3.
f1 = ( 2.0*vrat / ( 1.0 + vrat) )**pw
f2 = (( 2.0 / ( 1.0 + vrat ) )**pw) * (vrat**pw-1.0)

dirr="/Users/natashabatalha/Documents/eddysed/input/optics/"#"input/optics/"+this_gas+".refrind"


for irad in range(nrad):
    radius[irad,igas] = rmin * vrat**(float(irad)/3.)
    rup[irad,igas] = f1*radius[irad,igas]
    dr[irad,igas] = f2*radius[irad,igas]
    
    ## will be removed during packaging
for igas in range(ngas):
    this_gas=gas_name[igas]
    print(this_gas)
    filename = dirr +this_gas+".refrind"
    #put skiprows=1 in loadtxt to skip first line
    idummy, wave_in, nn, kk = np.loadtxt(open(filename,'rt').readlines(), unpack=True, usecols=[0,1,2,3])#[:-1]
    
    #convert to centimeters  
    wave= wave_in * 1e-4
        
    #do you need this??
    #idummy, wave_in, nn, kk = idummy[:nwave], wave_in[:nwave], nn[:nwave], kk[:nwave]
    
    for iwave in range(nwave):
        print(iwave)
        wvno=2*pi/wave[iwave] #moved this up here (no need to be in loop)
        
        for irad in [18]:#range(nrad):
            
            #subdivide radius grid into 6 bins to average out oscillations and call Mie 
            #code
            
            if irad== 0 :
                dr5= (( rup[0,igas] - radius[0,igas] ) / 5.)
                rr= radius[0,igas]
            else:
                dr5 = ( rup[irad,igas] - rup[irad-1,igas] ) / 5.
                rr  = rup[irad-1,igas]
            
            qext[iwave,irad,igas]=0.0
            qscat[iwave,irad,igas] = 0.0
            cos_qscat[iwave,irad,igas] = 0.0
        
            corerad = 0.
            corereal = 1.
            coreimag = 0.
        
            bad_mie= False
            
            #loop through subdivisions
        
            for isub in range(6):
                corerad = 0.0 * rr 
                corereal = 1#nn[iwave]
                coreimag =  0#kk[iwave] 

                arr=ps.MieQCoreShell( corereal+(1j)*coreimag, 
                                     nn[iwave]+(1j)*kk[iwave], 
                                     wave[iwave]*1e7,dCore=0,dShell=2.0*rr*1e7)
                
                
                qext[iwave,irad,igas]+= arr[0]
                qscat[iwave,irad,igas]+= arr[1]
                cos_qscat[iwave,irad,igas] += arr[3]
                
                rr+=dr5
                
            qext[iwave,irad,igas] = qext[iwave,irad,igas] / 6.
            qscat[iwave,irad,igas] = qscat[iwave,irad,igas] / 6.
            cos_qscat[iwave,irad,igas] = cos_qscat[iwave,irad,igas] / 6.

        

Mg2SiO4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195


In [4]:
mark =  pd.read_csv(os.path.join(dirr,this_gas+".mieff"), skiprows=1,names=['wave','qscat','qext','cos_qscat'], delim_whitespace=True)

radii = mark.loc[np.isnan(mark['qscat'])]['wave'].values
mark = mark.dropna()

wave_mark= mark['wave'].values.reshape((len(radii),196)).T
qscat_mark = mark['qscat'].values.reshape((len(radii),196)).T
qext_mark = mark['qext'].values.reshape((len(radii),196)).T
cos_qscat_mark = mark['cos_qscat'].values.reshape((len(radii),196)).T



In [5]:
fscat = figure(height=300, width=800, x_axis_type='log', y_axis_type='log',y_axis_label='Scattering')
fext = figure(height=300, width=800, x_axis_type='log', y_axis_type='log',y_axis_label='Extinction')
fasy = figure(height=300, width=800, x_axis_type='log',  x_axis_label='Micron')

rads = [18]
cols = plasma(len(rads))
for r,c in zip(rads,cols):
    fscat.line(1e4*wave, qscat[:,r,0],color=c,legend='New Py r='+str(int(radii[r]*1e7)/1e3)+'um')
    fext.line(1e4*wave, qext[:,r,0],color=c,legend='New Py r='+str(int(radii[r]*1e7)/1e3)+'um')

    fscat.circle(1e4*wave_mark[:,r], qscat_mark[:,r],color=c,legend='Old Fortran')#,line_dash = 'dashed')
    fext.circle(1e4*wave_mark[:,r], qext_mark[:,r],color=c,legend='Old Fortran')#,line_dash = 'dashed')

    fasy.circle(1e4*wave, abs(cos_qscat_mark[:,r]/qscat_mark[:,r]) ,legend='Old Fortran',color=c)
    fasy.line(1e4*wave, abs(cos_qscat[:,r,0]) ,legend='New Py r='+str(int(radii[r]*1e7)/1000.0)+'um',color=c)
show(column(fscat, fext, fasy))


